# Introducción al Aprendizaje Automático

### Motivación:


Se desea estimar el monto de compra de un usuario.

### Consignas:


0. Cargar los datos.
1. Seleccionar y transformar los features que consideren relevantes para dicha tarea.
2. Realizar el split de los datos correspondiente.
3. Seleccionar uno o dos modelos acordes para la tarea.
4. Seleccionar métricas.
5. Realizar el entrenamiento.
6. Evaluación. Realizar cross-validation.
7. Conclusiones.

HINTS: Probar feature scaling, ajuste de hiperparámetros, agregar/quitar features, etc.

Si un modelo ya da medio bajo, capaz no hay que seguir probando mucho con ese y probar con otro.

Ver cuáles son los features interesantes para predecir el valor de compra.
Usar modelos muy basicos -> ver primero algo lineal, y después ver algo más. Emi probo lo lineal y no andaba muy bien, pero servia.

1) Seleccionar y transformar los features. Los features hay que codificarlos, a los categoricos. El problema es que el modelo va a tomar a las categorias con números más altos con mayor preponderancia. One-J-encoding puede ser o también normalizarlos capaz.Probar con dos o tres columnas de features, y ver despues si el modelo mejora o empeora agregando mas features.
NO ASIGNAR NUMEROS ASI NOMAS A LAS CATEGORIAS. Esto lo soluciono con One-J-Encoding, que cada variable va a tener un vectorcito con distinta dimensionalidad, y uno que se mantiene. 
Por cada uno de los features me construye un vector (en categoria de ciudad, me va a ser 3 columnas A, B y C, en donde me pone un 1 en la columna que corresponda.
Y asi va a ser para cada variable. Cada feature va a tener sus diferentes columnas. La cardinalidad va a ser la suma de las columnas para cada variable. Va a ser una matriz , y a esa matriz le aplico un metodo de scikylearn que me lo normaliza. (creo que toma la media menos la desviacion estandard).
Entonces, va a ser una matriz con por ejemplo, 3 columnas donde van a ser las categorias de ciudad, más 10 columnas ponele que son los tipos de categoria de producto, y todas las columnas van a tener la misma cantidad de filas, que es el tamaño del dataset. 
Luego sobre esta matriz de One-J-Encoding aplico PCA. 

Otra forma es asignar numeros a las categorias, y despues normalizarlo. 

4) Ver las metricas, para ver que tan bueno es. En este problema hay al menos 3 metricas claras

6) Cross validation

HINTS -> probar feature scaling, ver si vale la pena normalizar los features o no.
Ajuste de hiperparámetros, depende del modelo que usemos. 
Probar con dos a ojo nomás, y despues ir agregando más features.



Link para ver los distintos tipos de métricas (funciones de costo) para regresión

https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0

In [37]:
import numpy
import pandas
import seaborn
import scipy
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron, Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

In [2]:
ventas = pandas.read_csv(r"C:\Users\Usuario\Desktop\FACULTAD\Diplomatura FAMAF\Mentoria - Estimacion de ventas\Materia 1 - Analisis y visualizacion\Dataset\retail_sales.csv")
ventas

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,83.70
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,152.00
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,14.22
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,10.57
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,79.69
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,152.27
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,192.15
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,158.54
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,156.86
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,78.71


In [4]:
pandas.DataFrame(ventas.groupby('Gender').describe()[['Purchase']])  

Purchase                                                          
           count       mean        std   min    25%    50%     75%     max
Gender                                                                    
F       132197.0  88.097613  47.214401  1.85  54.59  79.29  114.36  239.59
M       405380.0  95.047717  50.510573  1.85  59.01  81.12  125.42  239.61

In [7]:
pandas.DataFrame(ventas.groupby('City_Category').describe()[['Purchase']])

Purchase                                                      \
                  count       mean        std   min    25%    50%       75%   
City_Category                                                                 
A              144638.0  89.580110  48.668966  1.85  54.19  79.41  118.0175   
B              226493.0  91.986578  49.270630  1.85  54.77  80.15  120.0000   
C              166446.0  98.444419  51.094721  1.86  60.94  86.18  132.6800   

                       
                  max  
City_Category          
A              239.61  
B              239.60  
C              239.61

In [8]:
pandas.DataFrame(ventas.groupby('Marital_Status').describe()[['Purchase']])  ## Me parece que no me va a servir esta variable

Purchase                                                    \
                   count       mean        std   min    25%    50%     75%   
Marital_Status                                                               
0               317817.0  93.333255  49.852539  1.85  58.50  80.59  120.80   
1               219760.0  93.346327  49.749070  1.86  58.86  80.66  120.65   

                        
                   max  
Marital_Status          
0               239.61  
1               239.61

In [9]:
pandas.DataFrame(ventas.groupby('Occupation').describe()[['Purchase']])  ## Me parece que no va a servir

Purchase                                                         \
              count       mean        std   min      25%     50%       75%   
Occupation                                                                   
0           68120.0  91.869467  49.352749  1.88  54.7000  80.160  119.7900   
1           45971.0  90.177031  47.913294  1.85  58.5900  79.770  117.2500   
2           25845.0  90.259390  49.106588  1.97  54.4100  79.640  117.5300   
3           17366.0  92.380773  49.633098  1.88  58.3400  80.210  119.7775   
4           70862.0  92.790267  50.036969  1.85  54.6300  80.580  120.5200   
5           11985.0  93.888490  49.876674  2.09  54.7600  80.920  121.1100   
6           19822.0  93.363786  49.478513  1.88  59.2325  80.675  120.0275   
7           57806.0  95.021753  50.442302  1.87  59.1400  80.850  126.0875   
8            1524.0  95.765085  48.846318  1.93  59.9575  85.405  126.4200   
9            6153.0  87.143359  46.001949  2.09  54.2700  79.020  114.6200   
10          12623.0  90.528364  50.735368  1.87  53.6400  80.490  119.7450   
11          11338.0  92.994672  50.557929  1.85  58.8900  80.620  120.3375   
12          30423.0  98.830525  50.835330  1.87  60.9800  85.960  132.7600   
13           7548.0  94.244494  48.688158  1.93  60.8475  81.110  118.5700   
14          26712.0  95.685364  50.219508  1.97  59.5500  81.370  125.7550   
15          11812.0  98.662399  50.399092  2.00  61.5075  85.840  132.1300   
16          24790.0  94.571331  49.518256  1.99  59.4600  80.790  122.9800   
17          39090.0  99.063790  50.836333  1.86  60.5500  86.620  133.5850   
18           6525.0  92.336714  49.372633  2.06  54.4100  79.660  120.7700   
19           8352.0  87.542492  49.995202  1.85  53.1075  78.510  117.5725   
20          32910.0  88.810995  48.906447  1.86  54.0500  79.155  116.9600   

                    
               max  
Occupation          
0           239.61  
1           239.60  
2           239.55  
3           239.14  
4           239.61  
5           239.24  
6           239.51  
7           239.48  
8           238.69  
9           239.43  
10          239.55  
11          239.46  
12          239.60  
13          239.59  
14          239.41  
15          239.49  
16          239.47  
17          239.61  
18          238.94  
19          239.39  
20          239.60

In [10]:
pandas.DataFrame(ventas.groupby('Age').describe()[['Purchase']])  ## Puede llegar a servir

Purchase                                                            
          count       mean        std   min    25%    50%       75%     max
Age                                                                        
0-17    14707.0  90.201269  50.604989  1.87  53.65  80.09  119.0450  239.55
18-25   97634.0  92.351976  49.957234  1.85  54.39  80.41  120.4400  239.58
26-35  214690.0  93.145890  49.740814  1.85  58.33  80.43  120.6300  239.61
36-45  107499.0  94.014788  49.782521  1.85  59.10  80.76  121.3100  239.60
46-50   44526.0  92.848723  49.212536  1.86  59.21  80.50  120.2175  239.60
51-55   37618.0  96.206166  50.353265  1.87  60.56  81.72  125.7850  239.60
55+     20903.0  94.538986  49.391245  1.87  60.68  81.27  119.7600  239.60

In [11]:
pandas.DataFrame(ventas.groupby('Stay_In_Current_City_Years').describe()[['Purchase']])  ## No creo que me sirva


Purchase                                     \
                               count       mean        std   min    25%   
Stay_In_Current_City_Years                                                
0                            72725.0  92.472386  49.519215  1.85  58.42   
1                           189192.0  93.198651  49.840099  1.85  58.48   
2                            99459.0  93.976073  49.988867  1.85  58.92   
3                            93312.0  93.506851  49.798788  1.87  58.72   
4+                           82889.0  93.463702  49.783474  1.90  58.87   

                                                    
                               50%     75%     max  
Stay_In_Current_City_Years                          
0                           80.380  120.11  239.60  
1                           80.550  120.60  239.61  
2                           80.890  121.88  239.61  
3                           80.605  120.92  239.61  
4+                          80.670  120.58  239.58

In [12]:
pandas.DataFrame(ventas.groupby('Product_Category_1').describe()[['Purchase']])  ## Si o si me sirve

Purchase                                                   \
                       count        mean        std    min       25%      50%   
Product_Category_1                                                              
1                   138353.0  136.077015  42.975999  37.90  115.4600  152.450   
2                    23499.0  112.556808  35.693635  31.76   96.4700  127.300   
3                    19849.0  100.968417  28.262172  26.38   81.9800  107.420   
4                    11567.0   23.288629   8.133716   6.84   20.5800   21.750   
5                   148592.0   62.380040  19.094178  17.13   52.4100   69.110   
6                    20164.0  158.378936  40.109383  39.81  125.0300  163.120   
7                     3668.0  163.738302  41.686116  40.61  128.4875  167.010   
8                   112132.0   74.973548  20.135495  19.39   60.3600   79.050   
9                      404.0  155.382970  53.054835  45.28  135.8650  143.885   
10                    5032.0  196.799744  42.332826  46.24  185.4500  192.055   
11                   23960.0   46.829336  18.347002  14.72   31.3000   46.100   
12                    3875.0   13.511956   3.620882   3.42   10.7100   14.010   
13                    5440.0    7.226195   1.835191   1.85    5.7800    7.550   
14                    1500.0  131.454520  40.677960  36.57  110.9750  146.555   
15                    6203.0  147.764222  51.781194  41.48  125.2050  166.600   
16                    9697.0  147.641575  43.617143  40.36  123.5300  162.910   
17                     567.0  101.564409  23.399859  26.16   80.6100  104.350   
18                    3075.0   29.753076   7.243014   7.54   23.5900   30.720   

                                      
                         75%     max  
Product_Category_1                    
1                   158.1200  197.08  
2                   132.1300  165.04  
3                   132.1200  137.17  
4                    28.3700   35.56  
5                    71.5600   89.07  
6                   200.5200  206.90  
7                   204.8600  210.80  
8                    97.2100  100.82  
9                   187.6400  235.31  
10                  234.3800  239.61  
11                   60.5700   76.54  
12                   17.2300   17.78  
13                    9.2700    9.62  
14                  151.7725  189.31  
15                  207.4500  215.69  
16                  168.3200  209.71  
17                  127.7200  132.64  
18                   37.6900   39.00

In [3]:
## De esta manera, hago que el sistema no asuma ningún orden entre las categorías.
## De la columna estado que tiene 6 categorías, me hace 6 columnas distintas, y me pone un 1 cuando corresponde a esa categoría.


## Otra forma de hacerlo es con -> pd.get_dummies(data)  !!!!!!!

from sklearn.preprocessing import LabelBinarizer


other_column = 'City_Category'

lb = LabelBinarizer()

lb_results = lb.fit_transform(ventas[other_column])
categoria_ciudad = pandas.DataFrame(lb_results, columns=((other_column + '_') + pandas.Series(lb.classes_)))


In [4]:
other_column = 'Age'

lb = LabelBinarizer()

lb_results = lb.fit_transform(ventas[other_column])
edad = pandas.DataFrame(lb_results, columns=((other_column + '_') + pandas.Series(lb.classes_)))

In [5]:
other_column = 'Gender'

lb = LabelBinarizer()

lb_results = lb.fit_transform(ventas[other_column])
genero1 = numpy.hstack((lb_results, 1 - lb_results))
genero = pandas.DataFrame(genero1, columns=(pandas.Series(lb.classes_)))

#genero = pandas.DataFrame(lb_results, columns=(pandas.Series(lb.classes_)))

In [6]:
other_column = 'Product_Category_1'

lb = LabelBinarizer()

lb_results = lb.fit_transform(ventas[other_column])
categoria_producto = pandas.DataFrame(lb_results, columns=(pandas.Series(lb.classes_)))

In [7]:
other_column = 'Occupation'

lb = LabelBinarizer()

lb_results = lb.fit_transform(ventas[other_column])
ocupacion = pandas.DataFrame(lb_results, columns=(pandas.Series(lb.classes_)))

In [8]:
precio = pandas.DataFrame(ventas.Purchase)


In [18]:
## Probamos utilizando las variables Edad y Categoria de Producto, ambas codificadas


merge = pandas.merge(categoria_ciudad, categoria_producto, right_index=True, left_index=True)
merge = pandas.merge(merge, edad, right_index=True, left_index=True)
ventas_encod = pandas.merge(precio, merge, right_index=True, left_index=True)




In [39]:
# División entre instancias y etiquetas
X, y = ventas_encod.iloc[:, 1:], ventas_encod.Purchase

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [41]:
## REGRESIÓN LINEAL

## Mejores combinaciones
#  1. Categoria ciudad, categoria producto, y edad
#  2. Categoria ciudad, categoria producto, y ocupacion
#

# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_test, model.predict(X_test)))
print()
print('Media del error absoluto para entrenamiento: %.2f' %
      mean_absolute_error(y_train, model.predict(X_train)))
print('Media del error absoluto para validación: %.2f' %
      mean_absolute_error(y_test, model.predict(X_test)))

# plt.figure(figsize=(14, 5), dpi= 80, facecolor='w', edgecolor='k')

# X_range_start = numpy.min(numpy.r_[X_train, X_test])
# X_range_stop = numpy.max(numpy.r_[X_train, X_test])
# y_range_start = numpy.min(numpy.r_[y_train, y_test])
# y_range_stop = numpy.max(numpy.r_[y_train, y_test])
# X_linspace = numpy.linspace(X_range_start, X_range_stop, 200).reshape(-1, 1)

# # Conjunto de entrenamiento
# plt.subplot(1, 2, 1)
# plt.scatter(X_train, y_train, facecolor="dodgerblue", edgecolor="k", label="datos")
# plt.plot(X_linspace, model.predict(X_linspace), color="tomato", label="modelo")
# plt.ylim(y_range_start, y_range_stop)
# plt.title("Conjunto de Entrenamiento")

# # Conjunto de validación
# plt.subplot(1, 2, 2)
# plt.scatter(X_test, y_test, facecolor="dodgerblue", edgecolor="k", label="datos")
# plt.plot(X_linspace, model.predict(X_linspace), color="tomato", label="modelo")
# plt.ylim(y_range_start, y_range_stop)
# plt.title("Conjunto de Validación")

Media del error cuadrado para entrenamiento: 919.50
Media del error cuadrado para validación: 905.61

Media del error absoluto para entrenamiento: 23.04
Media del error absoluto para validación: 22.89


In [19]:
## REGRESIÓN POLINOMIAL


polynomial_degree = 2# TODO: Probar distintos grados del polinomio

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(X_train)
X_poly_train = poly_features.transform(X_train)
X_poly_test = poly_features.transform(X_test)

model = LinearRegression()
model.fit(X_poly_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_test, model.predict(X_poly_test)))
print()
print('Media del error absoluto para entrenamiento: %.2f' %
      mean_absolute_error(y_train, model.predict(X_train)))
print('Media del error absoluto para validación: %.2f' %
      mean_absolute_error(y_test, model.predict(X_test)))
# mce_train = []
# mce_valid = []

# for x in range(2,8,2):
#     print('Grado del polinomio: ', x)
#     poly_features = PolynomialFeatures(x)
    
#     poly_features.fit(X_train)
#     X_poly_train = poly_features.transform(X_train)
#     X_poly_test = poly_features.transform(X_test)

#     model = LinearRegression()
#     model.fit(X_poly_train, y_train)

#     print('Media del error cuadrado para entrenamiento: %.2f' % 
#       mean_squared_error(y_train, model.predict(X_poly_train)))
#     print('Media del error cuadrado para validación: %.2f' %
#       mean_squared_error(y_test, model.predict(X_poly_test)), '\n') 

#     mce_train.append(mean_squared_error(y_train, model.predict(X_poly_train)))
#     mce_valid.append(mean_squared_error(y_test, model.predict(X_poly_test)))


Media del error cuadrado para entrenamiento: 915.04
Media del error cuadrado para validación: 901.80


In [35]:
## Probamos con PCA,y luego aplicamos los modelos

pca = PCA(n_components = 2)
X,y = ventas_encod.iloc[:, 2:], ventas_encod.Purchase
ventas_pca = pca.fit_transform(X) 
X_train, X_test, y_train, y_test = train_test_split(ventas_pca, y, test_size=0.2, random_state=0)



In [36]:
## Probamos el modelo lineal con PCA. Da bastante feo

model = LinearRegression()
model.fit(X_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_test, model.predict(X_test)))
print()
print('Media del error absoluto para entrenamiento: %.2f' %
      mean_absolute_error(y_train, model.predict(X_train)))
print('Media del error absoluto para validación: %.2f' %
      mean_absolute_error(y_test, model.predict(X_test)))


Media del error cuadrado para entrenamiento: 2466.02
Media del error cuadrado para validación: 2452.29
